# Naturalistic EMA validation

Applying the findings from the 4-state correlation work (EMA x UPDRS) onto real-life EMA data.

Goals:
- analyse real-life variation of EMA values
    - inter-individual variation
    - intra-individual variation, daily fluctuations, differences between days

## 0. Import packages

- document versions for reproducibility

In [ ]:
# import packages
import datetime as dt
import pandas as pd
import numpy as np
import os
import sys
import csv
import json
import importlib
from itertools import product, compress
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, variation

from dataclasses import dataclass, field


In [ ]:
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
# print('sci-py', scipy.__version__)
# print('sci-kit learn', sk.__version__)
# print('matplotlib', plt_version)

"""
Python sys 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
pandas 2.1.1
numpy 1.26.0

from 16.09

Python sys 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
pandas 2.3.2
numpy 2.3.3
"""

Import custom functions

In [ ]:
# from dbs_home repo
from dbs_home.load_raw.main_load_raw import loadSubject 
import dbs_home.utils.helpers as home_helpers
import dbs_home.utils.ema_utils as home_ema_utils
import dbs_home.plot_data.plot_compliance as plot_home_compl
import dbs_home.preprocessing.preparing_ema as home_ema_prep

In [ ]:
# from current repo
from utils import load_utils, load_data, prep_data
from plotting import plot_help



## 1. Import Home-Data

Use pre-operative sessions

- use 9-point-converter
- use direct. inverter

Import EMA home data from raw files

In [ ]:
MOMENTS = ['pre-op', 'pre 3MFU', 'post 3MFU']

sub_skip = [] # ['hm25',]  # skip full subject
# skip per session
ses_skip = [['hm20', 'ses03'],]
# ses_skip = [['hm14', 'ses03']]

In [ ]:
sessions_include = {m: {} for m in MOMENTS}

for rec_moment in MOMENTS:

    sel_info = home_helpers.select_sessions(target_session=rec_moment)
    sel_info = sel_info.set_index(sel_info['study_id'],)
    sel_sessions = {sub: ses for sub, ses in sel_info[['study_id', 'Session']].values}

    for key, val in sel_sessions.items():
        sessions_include[rec_moment][key] = val


In [ ]:
print(sessions_include.keys())
print(sessions_include)

Load EMA - data

In [ ]:
importlib.reload(home_ema_utils)
importlib.reload(load_data)
importlib.reload(prep_data)
importlib.reload(home_ema_prep)

# load all combined data

# SUBS_INCL = ['hm14']


data = {m: {} for m in MOMENTS}

for rec_moment, sub_sess in sessions_include.items():
    # rec_moment contains 'pre-op', or 'pre 3MFU', 'post 3MFU', etc
    for sub, ses in sub_sess.items():

        if sub in sub_skip: continue
                    
        if [sub, ses] in ses_skip: continue
                
        ses_class = loadSubject(
            sub=sub,
            ses=ses,
            incl_EMA=True,
            incl_ACC=False,
        )
        temp_df = home_ema_utils.load_ema_df(sub_ses_class=ses_class)
        # prepare
        temp_df = home_ema_prep.prepare_ema_df(temp_df, ADD_MEANMOVE=True, INVERT_NEG_ITEMS=False,)
        ### TODO: CHECK WHY NOT ALL SUBS ARE INVERTED PROPERLY

        data[rec_moment][sub] = temp_df


## 2. Explore naturalistic EMAs

Preprocess EMA

- merge scores
- invert negative-items (higher = clinically better)
- mean-correct EMA
    - test different normalizations:
        - normalize with grand-mean per sub
        - normalize with session mean

In [ ]:
importlib.reload(prep_data)

MOMENTS = ['pre-op', 'pre 3MFU', 'post 3MFU']

allsubs = []
for mom in list(data.keys()): allsubs.extend(list(data[mom].keys()))
allsubs = np.unique(allsubs)


corr_data = {m: {} for m in MOMENTS}

for sub in allsubs:

    subdf = home_ema_prep.merge_sub_ema_df(datadict=data, sub=sub)
    subdf = home_ema_prep.mean_correct_ema_df(subdf)

    # split and palce back as moment dfs    
    for moment in MOMENTS:
        corr_data[moment][sub] = subdf[subdf['moment'] == moment].reset_index(drop=True)



Visualize first EMA results full group

In [ ]:
PLOT_ITEMS = ['move_mean', 'walking', 'tremor']

PLOT_CORR = False
if PLOT_CORR: PLOT_DATADICT = corr_data
else: PLOT_DATADICT = data


fig, axes = plt.subplots(3, 1, figsize=(9, 6))
fname = 'motorItems_abs_perSub_perSes_1104'
if PLOT_CORR: fname = fname.replace('abs', 'corr')

x_margin = 2
bin_w = 0.5

fsize=14

x_starts = {list(data.keys())[0]: 0}  # first moment starts at 0

subcolors = plot_help.get_sub_colors(PLOT_DATADICT)


for i_ax, col in enumerate(PLOT_ITEMS):

    for i_mom, moment in enumerate(PLOT_DATADICT.keys()):

        if PLOT_CORR and i_mom == 0: col = f'{col}_corr'

        # loop over sub-dfs within moment and add specific column values
        list_values = [tempdf[col].values for tempdf in PLOT_DATADICT[moment].values()]
        box_subs = list(PLOT_DATADICT[moment].keys())  # subs included in this boxplot
        # sort by sub id
        i_sort = np.argsort(box_subs)
        box_subs = [box_subs[i] for i in i_sort]
        list_values = [list_values[i] for i in i_sort]

        # drop NaN values in lists
        list_values = [[v for v in l if not np.isnan(v)] for l in list_values]

        # plot boxes for one moment
        bp = axes[i_ax].boxplot(list_values, widths=bin_w,
                           positions=x_starts[moment] + bin_w * np.arange(len(list_values)),
                           patch_artist=True,)
        if i_ax == 0:
            if moment != list(PLOT_DATADICT.keys())[-1]:
                x_starts[list(PLOT_DATADICT.keys())[i_mom + 1]] = x_starts[moment] + len(list_values) * bin_w + x_margin

        # Loop over boxes
        for patch, patchsub in zip(bp['boxes'], box_subs):
            patch.set_facecolor(subcolors[patchsub])

    # pretty plot
    axes[i_ax].set_ylabel(f'{col}\n(EMA answer)', size=fsize,)

# pretty axes
for ax in axes:
    if not PLOT_CORR:
        ax.set_ylim(0, 10)
        ax.set_yticks(np.arange(1, 10, 2))
        ax.set_yticklabels(np.arange(1, 10, 2))
    else:
        ax.set_ylim(-5, 5)
        ax.set_yticks(np.arange(-4, 6, 2))
        ax.set_yticklabels(np.arange(-4, 6, 2))

    ax.set_xticks(list(x_starts.values()))
    ax.set_xticklabels(list(x_starts.keys()))
    ax.spines[['right', 'top']].set_visible(False)
    ax.tick_params(axis='both', labelsize=fsize, size=fsize,)

    ax.axhline(0, xmin=0, xmax=1,
               color='gray', alpha=.3, zorder=0,)
    if PLOT_CORR: ylines = [-4, -2, 2, 4,]
    else: ylines = [1, 3, 5, 7, 9]
    for yline in ylines:
        ax.axhline(yline, xmin=0, xmax=1, color='gray', alpha=.15, zorder=0,)

plt.tight_layout()

# plt.savefig(os.path.join(load_utils.get_onedrive_path('figures'),
#              'ema_naturalistic', fname),
#              dpi=300, facecolor='w',)

plt.close()

Check completion rates

In [ ]:


for rec_moment in data.keys():

    for sub in data[rec_moment].keys():

        df = data[rec_moment][sub]

        df['Submission'] = pd.to_numeric(df['Submission'], errors='coerce')
        rate = df['Submission'].mean()
        print(f"{sub} completion rate @ {rec_moment}: {rate:.0%}")



Sub-analysis paper: EMA pre-post vs UPDRS

In [ ]:
from statsmodels.regression.mixed_linear_model import MixedLM


In [ ]:
Q_SEL = 'general movement'

PER1 = 'pre-op'
PER2 = 'post 3MFU'

pt_pre = list(data[PER1].keys())
pt_sel = [p for p in list(data[PER2].keys()) if p in pt_pre]

stat_df = {'sub': [], 'period': [], 'ema': []}

pt_coding = {}

for i_pt, pt in enumerate(pt_sel):  # add index for pt coding
    pt_coding[pt] = i_pt
    for i_period, period in enumerate([PER1, PER2]):
        values = data[period][pt]
        values = values.loc[values['Submission'].astype(int) == 1]  # only take completed emas
        values = values[Q_SEL].astype(float)  # take selected question
        values = values[~np.isnan(values)]  # excl nan values
        stat_df['ema'].extend(values)
        stat_df['sub'].extend([i_pt] * len(values))
        stat_df['period'].extend([i_period] * len(values))

stat_df = pd.DataFrame(stat_df)

In [ ]:
for pt in np.unique(stat_df['sub']):

    temp_df = stat_df[stat_df['sub'] == pt]
    values = [temp_df[temp_df['period'] == p]['ema'] for p in [0, 1]]
    plt.boxplot(values)

    plt.close()

In [ ]:
### stats

# define model
lm_model = MixedLM(
    endog=np.array(stat_df['period']),  # dependent variable 
    exog=np.array(stat_df['ema']),  # independent variable (i.e., LID presence, movement)
    groups=np.array(stat_df['sub']),  # subjects
    exog_re=None,  # (None)  defaults to a random intercept for each group
)
# run and fit model
# try:
lm_results = lm_model.fit()
# except:
#     if allow_lm_error:
#         return False
#     else:
#         print(dep_var.shape, indep_var.shape, groups.shape)
#         lm_results = lm_model.fit()

# extract results
fixeff_cf = lm_results._results.fe_params[0]
pval = lm_results._results.pvalues[0]


In [ ]:
lm_results._results.fe_params, lm_results._results.pvalues

print(lm_results)

Check means and variances for movement items, tremor, and gait items
- split per sub
- split per ses

In [ ]:
temp_df.EMA_reports

In [ ]:
importlib.reload(home_helpers)

data = {}

# Define pre-operative sessions
sel_info = home_helpers.select_sessions()
sel_info = sel_info.set_index(sel_info['study_id'],)
sel_sessions = {sub: ses for sub, ses in sel_info[['study_id', 'Session']].values}
print(sel_sessions)


for sub, ses in sel_sessions.items():

    data[sub] = loadSubject(
        sub=sub,
        ses=ses,
        incl_EMA=True,
        incl_ACC=False,
    )



## 3. Explore naturalistic ACC and Extract Features

include loading option for ACC only for EMA windows, store these selected windows separately, to prevent loading of full acc data

In [ ]:
from dbs_home.preprocessing import acc_preprocessing as acc_prep
from dbs_home.preprocessing import get_submovements
import dbs_home.preprocessing.submovement_processing as submove_proc
import dbs_home.load_raw.load_watch_raw as load_watch

# current repo imports
import utils.acc_features as acc_fts
import utils.feat_extraction as ft_extr
import utils.data_handling_ema_acc as data_handling


In [ ]:
feas_data_path = os.path.join(
    os.path.dirname(load_utils.get_onedrive_path()),
    'PROJECTS', 'home_feasibility'
)
feas_fig_path = os.path.join(
    load_utils.get_onedrive_path('figures'),
    'feasibility'
)

ntrl_fig_path = load_utils.get_onedrive_path('emaval_fig')

#### Load ACC data
- create SVM
- filter data within the dataclass

In [ ]:
# import naturalistic data via dbs_home repo

# LID
sub_id = 'hm24'
ses_id = 'ses01'

# # tremor check for Anna
# sub_id = 'hm22'
# ses_id = 'ses01'

### test days for hm24-ses01  # dyskinesia
# dev_day_selection = ['2025-07-17', '2025-07-18']
# dev_day_selection = [f'2025-07-{d}' for d in np.arange(17, 31)]
dev_day_selection = []

### test days for hm20-ses01  # tremor
# dev_day_selection = [
#     '2025-06-13', '2025-06-14',
#     '2025-06-15', '2025-06-16'
# ]

home_dat = loadSubject(
    sub=sub_id,
    ses=ses_id,
    incl_STEPS=False,
    incl_EPHYS=False,
    incl_EMA=True,
    incl_ACC=True,
    day_selection=dev_day_selection
)

Check available EMAs

In [ ]:
plot_home_compl.plot_EMA_completion_perSession(home_dat)

#### Extract features from Acc-Windows aligned to EMAs

In [ ]:
importlib.reload(data_handling)
importlib.reload(ft_extr)


xy_dict = {}

iter_settings = {
    'nosm_allwindow':[False, True, True],
    'sm_merged': [True, False, False],
    'sm_single': [True, False, True]}


# EXTR_SM, SM_SINGLE = False, False
# EXTR_FULLWIN = True

for key, [EXTR_SM, EXTR_FULLWIN, SM_SINGLE] in iter_settings.items():

    xy_dict[key] = ft_extr.get_features_per_session(
        home_dat=home_dat,
        sub_id=sub_id,
        ses_id=ses_id,
        LOAD_SAVE_FEATS = True,
        # define how features should be extracted
        EXTRACT_FT_FROM_SMs = EXTR_SM,
        EXTRACT_FT_FULL_WIN = EXTR_FULLWIN,
        ACC_FEATS_on_SINGLE_MOVES = SM_SINGLE,
        STORE_SUBMOVES = False,
        # plotting settings
        SAVE_PLOT = False,
        SHOW_PLOT = False,
    )

In [ ]:
[f'{k}: {v.shape}, column-names: {list(v.keys())}' for k, v in xy_dict.items()]

#### feature extractions for full days

In [ ]:
importlib.reload(acc_prep)
importlib.reload(load_watch)
importlib.reload(ft_extr)


In [ ]:
# acc_dayClass = acc_prep.get_accel_day(dat=home_dat, day_string='2025-07-14',)

# day_dict_lists = acc_prep.get_day_ALL_AccWindows(
#     subSesClass=home_dat, str_day='2025-07-12',
#     START_HOUR=8, END_HOUR=22,
# )

In [ ]:
iter_settings = {
    'nosm_allwindow': {
        "EXTRACT_FT_FROM_SMs": False,
        "EXTRACT_FT_FULL_WIN": True,
        "ACC_FEATS_on_SINGLE_MOVES": False
    },
    'sm_merged': {
        "EXTRACT_FT_FROM_SMs": True,
        "EXTRACT_FT_FULL_WIN": False,
        "ACC_FEATS_on_SINGLE_MOVES": False
    },
    'sm_single': {
        "EXTRACT_FT_FROM_SMs": True,
        "EXTRACT_FT_FULL_WIN": False,
        "ACC_FEATS_on_SINGLE_MOVES": True
    }
}


ft_set_sel = 'sm_single'




temp_fts = ft_extr.get_features_per_session(
    ONLY_EMA_WINDOWS=False,
    home_dat=home_dat,
    sub_id=sub_id,
    ses_id=ses_id,
    LOAD_SAVE_FEATS = True,
    # define how features should be extracted
    STORE_SUBMOVES = False,
    # plotting settings
    SAVE_PLOT = False,
    SHOW_PLOT = False,
    **iter_settings[ft_set_sel],
)

# merge into one df
ft_full_ses = pd.DataFrame()

for d, f in temp_fts.items():
    ft_full_ses = pd.concat([ft_full_ses, f]).reset_index(drop=True)

ft_full_ses = ft_full_ses.set_index(keys="timestamp")

# check missings
# np.sum(pd.isna(ft_full_ses), axis=0), ft_full_ses.shape

In [ ]:
### identify and remove rows without submovements for submovement ft sets
if ft_set_sel.startswith('sm'):
    # select out windows without submoves
    NOMOVE_CRIT_1 = pd.isna(ft_full_ses['rms_acc_SMcfvar'])
    NOMOVE_CRIT_2 = ft_full_ses['rms_acc_SMmean'] == 0.0
    nomove_idx = np.where(np.logical_and(NOMOVE_CRIT_1, NOMOVE_CRIT_2))[0]
    move_idx = np.where(~np.logical_and(NOMOVE_CRIT_1, NOMOVE_CRIT_2))[0]
    nomove_times = ft_full_ses.index[nomove_idx]
    # only keep rows with submovements
    ft_full_ses = ft_full_ses.iloc[move_idx]


In [ ]:
ft_full_ses.shape, len(nomove_times)

Visualise and check submovement velocity and PCAs
- !! currently requires code parts that are internalised within ft_extr.get_features_per_session()

In [ ]:
from sklearn.decomposition import PCA


In [ ]:


# for i_sm, tempwin in enumerate(sm_win_data[:30]):

#     for att in ['pc1', 'pc2']:
#         plt.plot(tempwin.timestamps, getattr(tempwin, att), label=att,
#                 lw=3, alpha=.5,)

#     plt.plot(tempwin.timestamps, tempwin.svm, label='svm',)
#     plt.plot(tempwin.timestamps, tempwin.velocity.T,
#             label=[f'velo_{a}' for a in 'xyz'],
#             ls='--',)

#     plt.title(f'submovement # {i_sm}')
#     plt.legend()

#     plt.show()

In [ ]:


# plt.plot(tempwin.velocity.T)

# pca = PCA(n_components=2)
# projected = pca.fit_transform(tempwin.velocity.T)  # Shape: (N, 2)

# pc1 = projected[:, 0]  # Primary direction
# pc2 = projected[:, 1]  # Secondary direction

# plt.plot(pc1, alpha=.3, lw=5,)
# plt.plot(pc2, alpha=.3, lw=5,)

# plt.show()

In [ ]:
# plt.plot(np.abs(tempwin.velocity.T))

# pca = PCA(n_components=2)
# projected = pca.fit_transform(np.abs(tempwin.velocity.T))  # Shape: (N, 2)

# pc1 = projected[:, 0]  # Primary direction
# pc2 = projected[:, 1]  # Secondary direction

# plt.plot(pc1, alpha=.3, lw=5,)
# plt.plot(pc2, alpha=.3, lw=5,)

# plt.show()

visualise heartrate

In [ ]:
# importlib.reload(load_watch)

# hr_day_data = load_watch.get_source_heartrate_day(
#     sub=home_dat.sub, ses=home_dat.ses, date='2025-08-13',
# )

In [ ]:
# t1 = windat.acc_times[0]
# t2 = windat.acc_times[-1]

# hr_sel = np.logical_and(hr_day_data['timestamp'] > t1,
#                         hr_day_data['timestamp'] < t2)

# hr_win = hr_day_data[hr_sel].reset_index(drop=True)



# hr_win

In [ ]:
# fig, ax = plt.subplots(1, 1)

# ax2 = ax.twinx()

# ax.plot(windat.acc_times, windat.acc_svm)
# ax2.plot(hr_win['timestamp'], hr_win[' HeartRate'], color='orangered',)

# ax2.plot(hr_day_data['timestamp'], hr_day_data[' HeartRate'],
#          color='orangered',)


# plt.show()

check of class-based ft extraction

In [ ]:
importlib.reload(acc_fts)
importlib.reload(ft_extr)

for ft in FEATS_INCL:

    value = getattr(sm_ft_class, f'run_{ft}')()  # extra brackets () for executing function

    print(f'{ft}: {value}')



in case later necessary:

In [ ]:
def round_datetime(t, T_RES_Sec = .1):
    """rounds on 0.1 sec"""

    t_sec = t.microsecond / 1e6
    t_sec_round = round(t_sec, abs(np.log10(T_RES_Sec)).astype(int))
    micro = t_sec_round * 1e6

    # add full second if rounding goes to 1e6 microseconds
    if micro == 1e6:
        t = t.replace(microsecond=0)
        t += dt.timedelta(seconds=1)
    # else replace rounded microseconds
    else:
        t = t.replace(microsecond=int(micro))

    return t

## 4. Evaluate extracted Features

- Hssayeni et al, Scientific Reports 2021
    - strongest wrist-features: angular velocity, standard deviation, power of secondary frequency, power of 1–4 Hz band, and Shannon Entropy (r = 0.82  - r = 0.75)

- from svm: classic features
- include cross-corr between pc1 and pc2


Evaluate Features per window
- distinction between extraciton over merged-svm or per-submovement is done in extraction code above

In [ ]:
def normalize_values(
    values, ZSCORE=True, LOG=True, return_kept_idx=False,
    SET_Z_mean_std=None,
):

    # log transform    
    if LOG:
        # remove zeros and nans
        if return_kept_idx:
            kept_idx = np.where([~np.isnan(v) and v != 0.0 for v in values])[0]
        values = [v for v in values if ~np.isnan(v) and v != 0.0]
        values = np.log(values)
        
    # zscore
    if ZSCORE:
        if SET_Z_mean_std:
            zmean, zstd = SET_Z_mean_std
        else:
            zmean, zstd = np.nanmean(values), np.nanstd(values)
        values = (values - zmean) / zstd

    
    
    if return_kept_idx:
        return values, kept_idx
    else:
        return values

In [ ]:
MULTI_SESH = False
EXCL_HR = True

In [ ]:
# get array with shape n-windows, n-feats

if MULTI_SESH:
    ### MULTI-SESSION DATA
    X = np.concat([X_ALL['ses01'], X_ALL['ses02']]).T
    y = np.concat([y_ALL['ses01'], y_ALL['ses02']]).ravel()
else:
    ### SINGLE SESSION DATA
    X = np.array([l for l in list(FEAT_STORE.values())])
    # get array with length n-windows
    y = np.array(Y_STORE['LID']).astype(float)

print(X.shape)

# WITHOUT HR BCS OF NANS
if EXCL_HR:
    hr_sel = ['hr_' in k for k in list(FEAT_STORE.keys())]
    X = X[~np.array(hr_sel), :]
    FTNAMES_KEPT = [n for n in list(FEAT_STORE.keys()) if 'hr_' not in n]
else:
    FTNAMES_KEPT = list(FEAT_STORE.keys())


print(X.shape)


for i_ft, (ft_X, ft_name) in enumerate(zip(X, FTNAMES_KEPT)):
    # repeat y definition due to nan-dropping in normalization
    y_iter = y.copy()

    print(ft_X.shape, ft_name, len(y_iter))

    ft_X, kept_idx = normalize_values(ft_X, ZSCORE=True, LOG=True,
                                      return_kept_idx=True,
                                      SET_Z_mean_std=None,)
    print('kept:', len(kept_idx), 'total', len(y))
    y_iter = y_iter[kept_idx]

    fig, ax = plt.subplots(1,1, figsize=(8, 3))

    box_values = {str(y_value): [] for y_value in np.arange(1, 10)}

    for y_value in list(box_values.keys()):
        box_sel = y_iter == float(y_value)
        matching_values = ft_X[box_sel]
        matching_values = matching_values[~np.isnan(matching_values)]  # get rid of NaNs
        box_values[y_value].extend(matching_values)

    ax.boxplot(box_values.values())

    ax.set_xticklabels(list(box_values.keys()))

    ax.set_ylabel(ft_name)
    ax.set_title(ft_name)
    ax.set_xlabel('EMA Dyskinesia severity (Likert points)')


    plt.show()

In [ ]:
# for c, m in zip(FEAT_STORE['sm_count'], FEAT_STORE['sm_duration_mean']):
#     print(f"# {c},\t\t{round(c * m / 6, 1)} segments,\t\tmean: {round(m, 1)} s")

Check feature distributions
- pre and post log-transform and z-scoring

In [ ]:
# # get array with shape n-windows, n-feats
# X = np.array([l for l in list(FEAT_STORE.values())])
# # get array with length n-windows
# y = np.array(Y_STORE['LID']).astype(float)

ft_set = list(xy_dict.keys())[2]

skip_feats = ['timestamp', ]

for ftname, ftvalues in xy_dict[ft_set].items():

    if ftname in skip_feats: continue

    fig, axes = plt.subplots(1, 2, figsize=(8, 3))

    axes[0].hist(ftvalues)
    axes[0].set_ylabel('count (n)')
    axes[0].set_xlabel('raw values')

    # z-score and take log values
    # try:
    ftvalues = normalize_values(ftvalues, ZSCORE=True, LOG=True)
    # except:
    #     print(f'\n\n{ftname} FAILED:')
    #     print(ftvalues)
    axes[1].hist(ftvalues)
    axes[1].set_ylabel('count (n)')
    axes[1].set_xlabel('log. z-scored values')

    plt.suptitle(ftname)
    plt.tight_layout()

    plt.show()

#### make daycurves for full day features

In [ ]:
# check leftovers NaNs
np.sum(pd.isna(ft_full_ses), axis=0)

# drop heartrate columns
ft_full_ses = ft_full_ses.drop(columns=[k for k in ft_full_ses.columns if k.startswith('hr')])


In [ ]:
def get_dayminutes(t):

    if type(t) == str:
        t = dt.datetime.strptime(t[:16], "%Y-%m-%d %H:%M")

    minutes = t.minute
    minutes += (t.hour * 60)

    return minutes

In [ ]:
def get_daily_minutes_mask(HR_START=8, HR_END=22, WIN_LEN_minutes=15):

    # get day time raster (in minutes into day)
    t0 = dt.datetime.strptime(str(HR_START), "%H")
    n_wins = (HR_END - HR_START) * (60 / WIN_LEN_minutes)
    mask_dtimes = [t0 + dt.timedelta(minutes=int(WIN_LEN_minutes * i))
                   for i in np.arange(n_wins)]
    mask_minutes = [get_dayminutes(t) for t in mask_dtimes]

    return mask_minutes

In [ ]:
def get_ft_daily_mean(
    ft_values, ft_times,
    PLOT_SAMPLE_DISTRIBUTION=False,
):
    
    mask_minutes = get_daily_minutes_mask()
    mask_dict = {m: [] for m in mask_minutes}

    for i, v in enumerate(ft_values):
        # get daily minute of timestamp (df-index)
        t = get_dayminutes(ft_times[i])
        # add ft value to list of corresponding daily minute
        try:
            mask_dict[t].append(v)
        except KeyError:
            # daily minutes not correct, find closest
            i_alt = np.argmin(abs(np.array(mask_minutes) - t))
            min_alt = mask_minutes[i_alt]
            mask_dict[min_alt].append(v)
    
    daily_minutes = list(mask_dict.keys())
    daily_mean = np.array([np.nanmean(l) for l in mask_dict.values()])
    daily_std = np.array([np.nanstd(l) for l in mask_dict.values()])

    ### plot distribution of samples on daily-minutes-raster
    if PLOT_SAMPLE_DISTRIBUTION:
        for k, v in mask_dict.items():
            print(f'{k} min, {k / 60} h: {len(v)} samples')

        samples_at_minutes = [len(l) for l in mask_dict.values()]
        print(samples_at_minutes)
        fig,ax = plt.subplots(1,1, figsize=(8, 3))
        ax.bar(x=daily_minutes, height=samples_at_minutes,
               width=12, color='olivedrab', alpha=.3,)
        # ax.plot(daily_minutes, [len(l) for l in mask_dict.values()])
        ax.set_ylabel('n samples (count)')
        ax.set_xlabel('Time at Day (hours)')
        ax.set_xticks(daily_minutes[::8])
        ax.set_xticklabels((np.array(daily_minutes[::8])/60).astype(int),)

        plt.show()

    return daily_minutes, daily_mean, daily_std

In [ ]:
def plot_daily_ft_mean(
    daily_minutes, daily_mean, daily_std, ft_name,
    use_ax = None,
):

    plot_color = 'olivedrab'  # orchid, blue, olivedrab
    xtick_hop = 8

    if not use_ax:
        fig, ax = plt.subplots(1, 1, figsize=(8, 3))
        RETURN_AX = False
    else:
        ax = use_ax
        RETURN_AX = True

    ax.plot(daily_minutes, daily_mean, color=plot_color, lw=3,)
    ax.fill_between(daily_minutes, y1=daily_mean - daily_std,
                    y2=daily_mean + daily_std, alpha=.3,
                    color=plot_color,)

    ax.set_xticks(daily_minutes[::xtick_hop])
    ax.set_xticklabels((np.array(daily_minutes[::xtick_hop])/60).astype(int),)
    ax.set_xlabel('Time at Day (hours)')

    ax.set_ylabel(ft_name)

    if RETURN_AX:
        return ax
    else:
        plt.show()

In [ ]:
FT_SEL_LIST = [
    'sm_count',
    'rms_acc_SMmean',
    'pow_4_7_ratio_SMmean',
]

fig, axes = plt.subplots(len(FT_SEL_LIST), 1,
                         figsize=(8, 3*len(FT_SEL_LIST)))

for i_ax, FT_SEL in enumerate(FT_SEL_LIST):
    (
        daily_minutes, daily_mean, daily_std
    ) = get_ft_daily_mean(ft_full_ses[FT_SEL], ft_full_ses.index,)

    plot_daily_ft_mean(
        daily_minutes, daily_mean, daily_std, FT_SEL, use_ax=axes[i_ax]
    )

plt.tight_layout()

plt.close()

### Predict EMAs

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold


from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.metrics import (
    accuracy_score, r2_score,
    roc_auc_score, balanced_accuracy_score
)
from scipy.stats import f as stats_f
from sklearn.feature_selection import f_regression, r_regression


In [ ]:
def get_f_stat(y_pred, y_true, n_feats):
    # F-statistic for model usability
    
    # Sum of squares
    SSR = np.sum((y_pred - np.mean(y_true)) ** 2)   # Regression
    SSE = np.sum((y_true - y_pred) ** 2)   # Error
    SST = np.sum((y_true - np.mean(y_true)) ** 2)   # Total

    # Degrees of freedom
    df_reg = n_feats
    df_err = len(y_true) - (n_feats + 1)  # +1 for coeff next to betas of features

    # Mean squares
    MSR = SSR / df_reg if df_reg > 0 else np.nan
    MSE = SSE / df_err if df_err > 0 else np.nan
    F = MSR / MSE
    # p-value for the observed F
    f_pval = 1 - stats_f.cdf(F, df_reg, df_err)

    return F, f_pval

cross-validation n=1 concept

Define X and y
- clear NaNs

In [ ]:
def prep_X_y(
    X, y, REMOVE_ZERO_SUBMOVES=False, verbose=False,
    RETURN_EXCLUDED_NAN_BOOL=False,
):
    
    nan_rows = np.any(np.isnan(X), axis=1)

    if REMOVE_ZERO_SUBMOVES:
        zero_rows = np.any(X == 0.0, axis=1)
        nan_rows = np.logical_or(nan_rows, zero_rows)

    if verbose: print(f'removed window-rows bcs of NaNs: n={sum(nan_rows)}')
    X = X[~nan_rows]

    # double check nan removing
    check_nan_rows = np.any(np.isnan(X), axis=1)
    if verbose: print(f'after removing n-nanrows: n={sum(check_nan_rows)}')
    
    # z-score and log
    for i_col in np.arange(X.shape[1]):
        x_temp = X[:, i_col]
        x_temp = normalize_values(x_temp, ZSCORE=True, LOG=True,)
        X[:, i_col] = x_temp

    # get array with length n-windows
    if len(y) > 0: y = y[~nan_rows]
    else: y = None

    if not RETURN_EXCLUDED_NAN_BOOL: return X, y
    else: return X, y, nan_rows

In [ ]:
EXCL_HR = True
ZSCORE_Y = False

# # get array with shape n-windows, n-feats
# X = np.array([l for l in list(FEAT_STORE.values())]).T
# # get array with length n-windows
# y = np.array(Y_STORE['LID']).astype(float).reshape(-1, 1)

ft_set = list(xy_dict.keys())[2]
print(f'\fFeature set chosen: {ft_set}')

keys_excl_feats = [
    'timestamp', 'LID', 'tremor', 'overall_move', 'move_hands', 'wellbeing'
]
FTS_INCL = [k for k in list(xy_dict[ft_set])
            if (k not in keys_excl_feats) and ('hr' not in k)]
EMA_Y = 'LID'

if MULTI_SESH:
    ### MULTI-SESSION DATA
    X = np.concat([X_ALL['ses01'], X_ALL['ses02']]).T
    y = np.concat([y_ALL['ses01'], y_ALL['ses02']]).ravel()
else:
    ### SINGLE SESSION DATA
    X = xy_dict[ft_set][FTS_INCL].values.copy()
    y = xy_dict[ft_set][EMA_Y].values.astype(float)
    X, y = prep_X_y(X, y)  # delete nans, log, z-score


# # WITHOUT HR BCS OF NANS
# if EXCL_HR:
#     hr_sel = ['hr_' in k for k in list(FEAT_STORE.keys())]
#     X = X[~np.array(hr_sel), :]
#     FTNAMES_KEPT = [n for n in list(FEAT_STORE.keys()) if 'hr_' not in n]
# else:
#     FTNAMES_KEPT = list(FEAT_STORE.keys())

if ZSCORE_Y:
    y = ((y - np.mean(y)) / np.std(y))
    # y = (y - Z_y_mean) / Z_y_std


# plt.plot(y)
# plt.show()
# print(X.shape, y.shape)

# # WITHOUT HR BCS OF NANS
# if EXCL_HR:
#     hr_sel = ['hr_' in k for k in list(FEAT_STORE.keys())]
#     X = X[:, ~np.array(hr_sel)]
# print(X.shape, y.shape)

# check where nans are, TODO during creation
# list(compress(list(FEAT_STORE.keys()), np.any(np.isnan(X), axis=1)))

# if X.shape[0] > X.shape[1]:
#     nonnan_sel = ~np.any(np.isnan(X), axis=1)
#     y = y[nonnan_sel]
#     X = X[nonnan_sel, :]
# else:
#     nonnan_sel = ~np.any(np.isnan(X), axis=0)
#     X = X[:, nonnan_sel]
#     y = y[nonnan_sel]


print(X.shape, y.shape, len(FTS_INCL))


#### compare different feature sets (full-window vs single-movements extracted)

In [ ]:
from plotting import plot_home_preds
from utils import pred_nat_ema 

In [ ]:
for ft_set in list(xy_dict.keys()):

      print(f'\n{"#" * 5}\nFeature set chosen: {ft_set}')

      FTS_INCL = [k for k in list(xy_dict[ft_set])
                  if (k not in keys_excl_feats) and ('hr' not in k)]
      
      print(FTS_INCL)

In [ ]:
importlib.reload(plot_home_preds)
importlib.reload(pred_nat_ema)


### SELECT FEATURE SET for prediction

ft_set = list(xy_dict.keys())[2]
# print(f'\fFeature set chosen: {ft_set}')

keys_excl_feats = [
    'timestamp', 'LID', 'tremor', 'overall_move', 'move_hands', 'wellbeing'
]
EMA_Y = 'LID'


for ft_set in xy_dict.keys():

      if ft_set != 'sm_single': continue

      if ft_set.startswith('sm'): EXCL_ZEROS = True
      else: EXCL_ZEROS = False

      print(f'\n{"#" * 50}\nFeature set chosen: {ft_set}')

      FTS_INCL = [k for k in list(xy_dict[ft_set])
                  if (k not in keys_excl_feats) and ('hr' not in k)]

      X = xy_dict[ft_set][FTS_INCL].values.copy()
      y = xy_dict[ft_set][EMA_Y].values.copy().astype(float)
      # get day array for crossval leave day out
      days = xy_dict[ft_set]['timestamp']  # get timestamps per sample
      days = [d.split(' ')[0] for d in days]  # only take day string per timestamp
      day_codes = [np.where(np.unique(days) == d)[0][0] for d in days]  # convert strings into codes per day
      
      # TODO: include day in prep X y 
      X, y, excluded_bool = prep_X_y(
            X, y, REMOVE_ZERO_SUBMOVES=EXCL_ZEROS, verbose=False,
            RETURN_EXCLUDED_NAN_BOOL=True,
      )  # delete nans, log, z-score
      if sum(excluded_bool) > 0:
            day_codes = np.array(day_codes)[~excluded_bool]
            assert len(day_codes) == X.shape[0], 'incorrect day code length vs X'

      ### predict
      PERMS = False
      N_PERMS = 1000
      TO_PLOT = True

      CLS_METHOD = 'lasso'
      n_fold_cv = 3

      if not PERMS:
            y_pred_total = pred_nat_ema.classif_home_ema(
                  X=X, y=y, PERMS=PERMS, N_PERMS=N_PERMS, n_fold_cv=n_fold_cv,
                  CLS_METHOD=CLS_METHOD, ZSCORE_Y=ZSCORE_Y, verbose=False,
                  day_codes=day_codes, leave_day_out_cv=True,
            )

      else:
            print('\n\n ############\nADD PERMUTATION FLOW')


      #### PLOTTING
      if TO_PLOT:
            if MULTI_SESH:
                  FIGNAME = f'LidPred_{sub_id}_ses0102_{CLS_METHOD}_scatter_n{len(y)}'
            else:
                  FIGNAME = f'LidPred_{sub_id}_{ses_id}_{CLS_METHOD}_scatter_n{len(y)}'
            if EXCL_HR: FIGNAME += '_exclHR'
            else: FIGNAME += '_inclHR'

            plot_home_preds.scatter_preds(y=y, y_pred_total=y_pred_total,
                                          ZSCORE_Y=ZSCORE_Y, show=TO_PLOT,)



      

#### Train on EMA, test on whole session

Train

In [ ]:
CLS_METHOD = 'lasso'

CLS_LIB = {
    'linreg': LinearRegression(),
    'lda': LDA(),  # only applicable on categorical or binary values
    'lasso': Lasso(alpha=0.3)
}


FTS_INCL_train = [
    k for k in list(xy_dict[ft_set])
    if (k not in keys_excl_feats) and ('hr' not in k)
]


### define training X, y
X_ema_train = xy_dict[ft_set][FTS_INCL_train].values.copy()
y_ema_train = xy_dict[ft_set][EMA_Y].values.copy().astype(float)

X_ema_train, y_ema_train = prep_X_y(
    X_ema_train, y_ema_train,
    REMOVE_ZERO_SUBMOVES=EXCL_ZEROS, verbose=False,
)  # delete nans, log, z-score

# train model
clf = CLS_LIB[CLS_METHOD]
clf.fit(X_ema_train, y_ema_train)

### define test X
X_all_test = ft_full_ses.values.astype(float)
test_stamps = [
    dt.datetime.strptime(t[:19], "%Y-%m-%d %H:%M:%S")
    for t in ft_full_ses.index.values
]
X_all_test, _ = prep_X_y(
    X_all_test, y=[],
    REMOVE_ZERO_SUBMOVES=EXCL_ZEROS, verbose=False,
)  # delete nans, log, z-score

# compare features
assert list(ft_full_ses.keys()) == FTS_INCL, 'different feature sets'

test_pred = clf.predict(X_all_test)

In [ ]:
plt.plot(test_stamps, test_pred)

In [ ]:
### daily plot
(
    daily_minutes, daily_mean, daily_std
) = get_ft_daily_mean(test_pred, test_stamps,)

plot_daily_ft_mean(
    daily_minutes, daily_mean, daily_std, 'LID prediction', #use_ax=axes[i_ax]
)

Perm test

In [ ]:
# N_PERMS = 1000

# # TEST_SEL = daycode_arr > 20  # takes circa .33

# # X_train = X_arr[~TEST_SEL, :]
# # y_train = y_arr[~TEST_SEL]

# # # test cohort
# # X_test = X_arr[TEST_SEL, :]
# # y_true = y_arr[TEST_SEL]

# model = LDA()

# # Run prediction
# perm_mets ={'F': [], 'R': []}

# y_true = y_arr.astype(int)

# np.random.seed(27)

# for i_perm in np.arange(N_PERMS):

#     y_perm_pred = np.array([np.nan] * len(y_arr))

#     for day in np.unique(daycode_arr):

#         TEST_SEL = daycode_arr == day
#         X_train = X_arr[~TEST_SEL, :]
#         y_train = y_arr.astype(int)[~TEST_SEL]
#         np.random.shuffle(y_train)   

#         # test cohort
#         X_test = X_arr[TEST_SEL, :]

#         model = models[CLSF]
#         model.fit(X_train, y_train)

#         y_perm_pred[TEST_SEL] = model.predict(X_test)

#     # y_train = y_arr[~TEST_SEL]
#     # np.random.shuffle(y_train)   
#     # model.fit(X_train, y_train)

#     # y_perm_pred = model.predict(X_test)
#     # # np.random.shuffle(y_perm_pred)
#     y_perm_pred = np.array([np.round(v) for v in y_perm_pred])

#     F, f_pvalue = get_f_stat(y_pred=y_perm_pred, y_true=y_true,
#                              n_feats=X_test.shape[1])
#     prs_stat, _ = pearsonr(y_perm_pred, y_true)
#     perm_mets['F'].append(F)
#     perm_mets['R'].append(prs_stat)




In [ ]:
if MULTI_SESH:
    FIGNAME = f'LidPred_{sub_id}_SESS0102_{CLS_METHOD}_PermSign_n{N_PERMS}'
else:
    FIGNAME = f'LidPred_{sub_id}_{ses_id}_{CLS_METHOD}_PermSign_n{N_PERMS}'


fig, axes = plt.subplots(1, 2, figsize=(8, 3))

# pred_mets = {'F': pred_F, 'R': pred_corrcoef}

for i_ax, metr in enumerate(['F', 'R']):

    axes[i_ax].hist(perm_results[metr], color='gray', alpha=.5,)
    axes[i_ax].axvline(np.percentile(perm_results[metr], 97.5),
                       color='orange', alpha=.8, lw=3,
                       label=f'alpha=0.025\n(n-perm: {N_PERMS})',)
    
    axes[i_ax].axvline(perm_true_results[metr],
                       color='purple', alpha=.5, lw=1,
                       label='prediction',)
    
    p_calc = sum(perm_true_results[metr] < perm_results[metr]) / len(perm_results[metr])
    print(f'metric {metr}: p = {np.round(p_calc, 3)}')

    axes[i_ax].set_xlabel(f'{metr} score', size=14,)

    axes[i_ax].set_ylabel('count (n)', size=14)

    axes[i_ax].tick_params(axis='both', size=14, labelsize=14,)
    axes[i_ax].spines[['right', 'top']].set_visible(False)

axes[1].legend(frameon=False, fontsize=14,
               bbox_to_anchor=(.95, .5), loc='center left')

plt.tight_layout()

# plt.savefig(os.path.join(load_utils.get_onedrive_path('figures'),
#              'proof_kin_pred', FIGNAME),
#              dpi=300, facecolor='w',)

plt.show()